In [ ]:
# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

In [48]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [49]:
# Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.
df = pd.read_csv('/home/ilke/Documents/persona.csv')
df.head(10)

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17
5,29,android,male,bra,17
6,19,android,male,bra,17
7,59,android,male,bra,17
8,49,android,male,bra,17
9,19,android,male,bra,17


In [50]:
df.tail()

,PRICE,SOURCE,SEX,COUNTRY,AGE
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


In [51]:
df.shape

(5000, 5)

In [52]:
df.isnull().sum()  #eksik veri yok

PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [54]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


In [55]:
# Kaç unique SOURCE vardır? Frekansları nedir?
df["SOURCE"].nunique()

2

In [56]:
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [57]:
#Kaç unique PRICE vardır?
df["PRICE"].nunique()

6

In [58]:
#Hangi PRICE'dan kaçar tane satış gerçekleşmiş?
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [59]:
#Hangi ülkeden kaçar tane satış olmuş?
df.groupby('COUNTRY')['PRICE'].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

In [60]:
#Ülkelere göre satışlardan toplam ne kadar kazanılmış?
df.groupby('COUNTRY')['PRICE'].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [61]:
#SOURCE türlerine göre satış sayıları nedir?
df.groupby('SOURCE')['PRICE'].count()

SOURCE
android    2974
ios        2026
Name: PRICE, dtype: int64

In [62]:
#Ülkelere göre PRICE ortalamaları nedir?
df.groupby('COUNTRY')['PRICE'].mean()

COUNTRY
bra    34.327540
can    33.608696
deu    34.032967
fra    33.587459
tur    34.787140
usa    34.007264
Name: PRICE, dtype: float64

In [63]:
#SOURCE'lara göre PRICE ortalamaları nedir?
df.groupby('SOURCE')['PRICE'].mean()

SOURCE
android    34.174849
ios        34.069102
Name: PRICE, dtype: float64

In [64]:
#COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?
df.groupby(['COUNTRY', 'SOURCE']).agg({'PRICE': 'mean'})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [65]:
#COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?
df.groupby(['COUNTRY', 'SOURCE', 'SEX', 'AGE']).agg({'PRICE': 'mean'})

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

In [66]:
#Çıktıyı PRICE’a göre sıralayınız.
agg_df =df.groupby(['COUNTRY','SOURCE','SEX','AGE']).agg({'PRICE':'mean'})
agg_df =agg_df.sort_values('PRICE',ascending=False)
agg_df.head()   

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [67]:
#Indekste yer alan isimleri değişken ismine çeviriniz.
#• Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir. Bu isimleri değişken isimlerine çeviriniz.
agg_df = agg_df.reset_index() 

In [68]:
#Age değişkenini kategorik değişkene çeviriniz ve agg_df’e ekleyiniz.
print(agg_df['AGE'].max()) # AGE değişkenimizdeki maksimum yaşı bulduk.
agg_df['AGE_CAT'] = pd.cut(agg_df['AGE'], bins=[0, 18, 23, 30, 40, 66], 
                            labels=['0_18', '19_23', "24_30", "31_40", "41_66"])

66


In [69]:
#Yeni seviye tabanlı müşterileri (persona) tanımlayınız.
agg_df['customers_level_based'] = [str(row[0]).upper() + '_' + str(row[1]).upper() + 
                                    '_' + str(row[2]).upper() + '_' + str(row[5]).upper()for row in agg_df.values]
agg_df = agg_df[['customers_level_based','PRICE']] 


In [70]:
#Yeni müşterileri (personaları) segmentlere ayırınız.
#Yeni müşterileri (Örnek: USA_ANDROID_MALE_0_18) PRICE’a göre 4 segmente ayırınız.
#• Segmentleri SEGMENT isimlendirmesi ile değişken olarak agg_df’e ekleyiniz.
#• Segmentleri betimleyiniz (Segmentlere göre group by yapıp price mean, max, sum’larını alınız).
agg_df = agg_df.groupby('customers_level_based').agg({'PRICE':'mean'}).sort_values('PRICE', ascending=False)
agg_df = agg_df.reset_index()

In [71]:
agg_df['SEGMENT'] = pd.qcut(agg_df['PRICE'], 4, labels=['D', 'C', 'B', 'A'])

In [72]:
agg_df.groupby(["SEGMENT"]).agg({"PRICE": ["min", "max", "mean"]})

PRICE                      
               min        max       mean
SEGMENT                                 
D        19.000000  32.333333  29.206780
C        32.500000  34.077340  33.509674
B        34.103727  36.000000  34.999645
A        36.060606  45.428571  38.691234

In [89]:
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,FRA_ANDROID_FEMALE_24_30,45.428571,A
1,TUR_IOS_MALE_24_30,45.000000,A
2,TUR_IOS_MALE_31_40,42.333333,A
3,TUR_ANDROID_FEMALE_31_40,41.833333,A
4,CAN_ANDROID_MALE_19_23,40.111111,A


In [ ]:
#Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.
#• 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?
#• 35 yaşında IOS kullanan bir Fransız kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?

In [76]:
new_user = 'FRA_IOS_FEMALE_31_40'
agg_df[agg_df['customers_level_based'] == new_user]

,customers_level_based,PRICE,SEGMENT
78,FRA_IOS_FEMALE_31_40,32.818182,C


In [75]:
new_user1 = 'TUR_ANDROID_FEMALE_31_40'
agg_df[agg_df['customers_level_based'] == new_user1]

,customers_level_based,PRICE,SEGMENT
3,TUR_ANDROID_FEMALE_31_40,41.833333,A
